In [11]:
import sys

from confluent_kafka import Consumer, Producer, KafkaError
import socket


In [12]:
conf_p1 = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': socket.gethostname(),
}

producer = Producer(conf_p1)

conf_c1 = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'test',
    'enable.auto.commit': 'false',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(conf_c1)

In [13]:
topic = 'test'

In [14]:
def acked(err, msg):
    if err is not None:
        print("Failed to delivery message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced")

In [15]:
producer.produce(topic, key='key', value='value', callback=acked)

In [16]:
producer.flush()

Message produced


0

In [17]:
def consume_loop(consumer, topics, msg_process, min_commit_count):
    try:
        consumer.subscribe(topics)
        msg_count = 0
        while running:
            msg = consumer.poll(timeout=1.0)
            if msg is None:
                continue
            if msg.error():
                if msg.error().code == KafkaError._PARTITION_EOF:
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n'
                                     % (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    raise KafkaException(msg.error())
            else:
                msg_process(msg)
                msg_count += 1
                if msg_count % min_commit_count == 0:
                    consumer.commit(asynchrnous=True)
    finally:
        consumer.close()
